# **energyexplorer.io: data pipeline overview**

#### **Directions for navigating notebook**

Read section purpose and directions (if applicable) prior to running next code chunk.

### **Section 1. Get API Key from the National Renewable Energy Lab's (NREL) National Solar Radiation Database (NSRDB)**

#### **Purpose**

This notebook remotely accesses solar radiation datasets using NSRDB's API. These datasets are used to calculate the annual solar radiation potential at a user defined latitude and longitude. 

To recieve data from the NSRDB's API, an API Key must be passed along with your request for data.

#### **Directions**

1: Follow this [link](https://developer.nrel.gov/signup/) to sign up for an API Key (note: data hosting and access is a free service provide by NREL, no finanical information is required to get your key.) \
2: When you have acquired your API Key replace text: "ENTER YOUR NSRDB API KEY HERE" with the API Key. Keep the quotes surrounding your API Key. \
3: Run code chunk.

In [7]:
###############################################################
# API Key to access National Solar Radiation Database (NSRDB) #
###############################################################

class creds:
    api_key = "ENTER YOUR NSRDB API KEY HERE"

### **Section 2. Import packages**

#### **Purpose**

In this section, you will import packages used by the data pipeline to import data and conduct analysis. This interactive jupyter notebook is being hosted by a [JupyterLite](https://jupyterlite.readthedocs.io/en/latest/) server, a separate server from energyexplorer.io.

Because of this, we will need to install some packages that do not come natively with JupyterLite's kernel [Pyodide](https://pyodide.org/en/stable/). For a full list of packages included with Pyodide, [click here](https://github.com/jupyterlite/jupyterlite/tree/main/packages).

In [9]:
#####################################################################################
# Pyodide non-included packages install #############################################
#####################################################################################

%pip install NREL-PySAM
%pip install pycaiso
%pip install requests

########################################
# Pyodide non-included packages import #
########################################

# Import PySAM. This package provides python functions used to convert raw solar radiation values into energy generation based on the engineering parameters provided by the user
# Learn more about PySAM here: https://pypi.org/project/NREL-PySAM/

# Use site.addsitedir() to set the path to the SAM SDK API. Set path to the python directory.
import site
site.addsitedir('/Applications/sam-sdk-2015-6-30-r3/languages/python/')
import PySAM.PySSC as pssc

# Import pycasio. This package provides functions used to remotely access the California Independent System Operator's (CAISO) historical wholesale market price data.
# CAISO oversees the operation of California's bulk electric power system, transmission lines, and electricity market generated and transmitted by its member utilities.
# Learn more about CAISO here: https://www.caiso.com/Pages/default.aspx

import requests
from pycaiso.oasis import Node

########################################
# Pyodide included packages import #####
########################################

# General data wrangling

import pandas as pd
import numpy as np
import time
from datetime import datetime
from calendar import monthrange

# File management

import sys, os

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### **Section 3. Enter inputs for your proposed utility scale solar + battery plant into the dictionary object**

#### **Purpose**

In this section we define the parameters of our solar + battery plant. These parameters will be used throughout the data pipeline for energy and economic calculations.

#### **Directions**

Replace the default parameters with your project's proposed parameters for each ['key': value] pair (ex. 'installed_capacity': 100 -- 'installed_capacity' is the key and 100 is the value, change only the value). Replace only the value, do not alter the key. You may keep as many or all of the parameters as defaults. All parameters must be filled out.

In [2]:
user_inputs = {
    
    ####################################################
    # These parameters will be passed to the NSRDB API #
    ####################################################

    # unit: degrees | dtype: float | latitude of plant
    'project_latitude': 35.21803686349634,

    # unit: degrees | dtype: float | longitude of plant
    'project_longitude': -116.94075390362501, 

    # unit: years | dtype: int | lifetime of project
    'project_lifetime': 25, 

    # unit: MW | dtype: int | installed capacity of solar array
    'installed_capacity': 100,

    ########################################################################################
    # These parameters will be passed to pySAM to convert NSRDB data into solar generation #
    ########################################################################################

    # unit: none | dtype: float | ratio of installed DC capacity to the inverter's AC power rating - value should ideally be greater than 1 --> learn more here: https://tinyurl.com/49ufff42
    'dc_ac_ratio': 1.1,

    # unit: degrees | dtype: int | angle of solar array tilt relative to ground -- should be between 0 and 90 --> learn more here: https://tinyurl.com/y9v5nn3s
    'tilt': 25,

    # unit: degrees (assume north is 0 degrees) | dtype: int | angle of solar array, default value assumes due south facing
    'azimuth': 180,

    # unit: percent | dtype: int | efficiency of inverter (this is a measure of how much dc power will be converted to ac power)
    'inv_eff': 96,

    # unit: percent | dtype: float | percent of system losses
    'losses': 14.0757,

    # unit: none | dtype: int (0-4) | select solar array type (0=Fixed, 1=Fixed Roof, 2=1 Axis Tracker, 3=Backtracted, 4=2 Axis Tracker)
    'array_type': 0,

    # unit: none | dtype: float | ratio of photovoltaic area to total ground area (used to calculate total land area of plant, which is not used by energyexplorer.io) --> learn more here: https://tinyurl.com/muuwvhd2
    'gcr': 0.4,

    # unit: none | dtype: float | constant loss adjustment
    'adjust:constant': 0,

    #########################################################
    # These parameters will be passed to the battery script #
    #########################################################

    # unit: MWh | dtype: int | total energy capacity of on-site battery
    'battery_capacity': 400,

    # unit: MW | dtype: int | rated charge power (rate of energy transfer) of on-site battery
    'max_charge_rate': 100,

    # unit: MW | dtype: int | rated discharge power (rate of energy transfer) of on-site battery
    'max_discharge_rate': 100,

    ###################################################################
    # These parameters will be passed for final economic calculations #
    ###################################################################

    # unit: $ | dtype: int | capital cost per MWh of solar array and battery -- default value from https://www.nrel.gov/docs/fy22osti/80694.pdf does not include battery cost - page 19
    'capital_cost_per_mwh': 890000,

    # unit: $ | dtype: int | fixed cost per MWh of solar array and battery -- default value from https://www.nrel.gov/docs/fy22osti/80694.pdf does not include battery cost - page 19
    'fixed_per_mwh': 16060,

    # unit: % | dtype: int | discount rate for solar plant
    'real_discount_rate': 5,

    # unit: % | dtype: int | tax credit on capital cost of plant, in this case we assume 30% due to the passing of the IRA in 2022
    'tax_credit': 30,

    # unit: $/MWh | dtype: int | any additional subsidy per MWh of generation
    'subsidy': 0

    ###########################################################################################
    # Note: In this example notebook, we will assume the plant will be both solar + storage ###
    # and is selling to the open wholesale market (meaning they will recieve only), not a ppa #
    ###########################################################################################

    #'project_type': 'solar',
    #'contract_type': 'open_market',
    #'ppa_price': 35,
}

### **Section 4.a. Enter inputs for your proposed utility scale solar + battery plant into the dictionary object**

#### **Purpose**

Prepare input parameters for NSRDB API.

#### **Directions**

Assign values to first 3 variables under **User fills out** section then run code chunk.

In [8]:
################################################################
################ User fills out ################################
################################################################

# Your email address you used to sign up for API Key
your_email = 'ENTER YOUR EMAIL HERE' #example: youremail@ucsb.edu

# Your full name you used to sign up for API Key, use '+' instead of spaces.
your_name = 'ENTER YOUR NAME HERE' # example: 'Your+Name'

# Choose year of data using YYYY format (must keep quotes around year)
year = 'ENTER DATE HERE' # example: '2020'

################################################################
############### Skip the rest ##################################
################################################################

# Define the lat, long of the location and the year
lat, lon = user_inputs['project_latitude'], user_inputs['project_longitude']

# lat = input() #testing input function
# Pull api key object from creds.py
api = creds.api_key

# Set the attributes to extract (e.g., dhi, ghi, etc.), separated by commas.
attributes = 'ghi,dhi,dni,wind_speed,air_temperature,solar_zenith_angle'

# Set leap year to true or false. True will return leap day data if present, false will not.
leap_year = 'false'

# Set time interval in minutes, i.e., '30' is half hour intervals. Valid intervals are 30 & 60.
interval = '60'

# Specify Coordinated Universal Time (UTC), 'true' will use UTC, 'false' will use the local time zone of the data.
# NOTE: In order to use the NSRDB data in SAM, you must specify UTC as 'false'. SAM requires the data to be in the
# local time zone.
utc = 'false'

# Your reason for using the NSRDB.
reason_for_use = 'research'

# Your affiliation
your_affiliation = 'UCSB'

# Please join our mailing list so we can keep you up-to-date on new developments.
mailing_list = 'false'

### **Section 4.b. Pull only first 2 lines of dataset from NSRDB API and timezone and elevation as objects that will be later passed into SAM model**

#### **Directions**

Run code chunk

In [76]:
# create timezone and elevation objects for use in SAM model
# Declare url string
url = 'https://developer.nrel.gov/api/nsrdb/v2/solar/psm3-download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&full_name={name}&email={email}&affiliation={affiliation}&mailing_list={mailing_list}&reason={reason}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, name=your_name, email=your_email, mailing_list=mailing_list, affiliation=your_affiliation, reason=reason_for_use, api=api, attr=attributes)

# Return just the first 2 lines to get metadata:
info = pd.read_csv(url, nrows=1)

# Assign timezone and elevation objects
timezone, elevation = info['Local Time Zone'], info['Elevation']

In [77]:
# Return all but first 2 lines of csv to get data:
site_generation = pd.read_csv('https://developer.nrel.gov/api/nsrdb/v2/solar/psm3-download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&full_name={name}&email={email}&affiliation={affiliation}&mailing_list={mailing_list}&reason={reason}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, name=your_name, email=your_email, mailing_list=mailing_list, affiliation=your_affiliation, reason=reason_for_use, api=api, attr=attributes), skiprows=2)

# Set the time index in the pandas dataframe:
site_generation = site_generation.set_index(pd.date_range('1/1/{yr}'.format(yr=year), freq=interval+'Min', periods=525600/int(interval)))

# take a look
site_generation.head(25)

,Year,Month,Day,Hour,Minute,GHI,DHI,DNI,Wind Speed,Temperature,Solar Zenith Angle
2020-01-01 00:00:00,2020,1,1,0,30,0,0,0,1.1,3.7,165.15
2020-01-01 01:00:00,2020,1,1,1,30,0,0,0,1.2,3.1,155.28
2020-01-01 02:00:00,2020,1,1,2,30,0,0,0,1.4,2.3,143.48
2020-01-01 03:00:00,2020,1,1,3,30,0,0,0,1.4,1.9,131.27
2020-01-01 04:00:00,2020,1,1,4,30,0,0,0,1.3,2.1,119.07
2020-01-01 05:00:00,2020,1,1,5,30,0,0,0,1.0,2.6,107.13
2020-01-01 06:00:00,2020,1,1,6,30,0,0,0,0.8,3.0,95.65
2020-01-01 07:00:00,2020,1,1,7,30,67,29,417,0.9,4.1,84.75
2020-01-01 08:00:00,2020,1,1,8,30,243,46,770,1.2,6.2,75.16
2020-01-01 09:00:00,2020,1,1,9,30,399,57,880,1.6,8.8,67.11


### **Section 5. Pass NSRDB dataset to SAM and calculate hourly energy generation at plant location**

#### **Directions**

Run code chunk and view total energy generation for year, capacity factor, and hourly generation in Generation (MW) column.

In [78]:
ssc = pssc.PySSC()

# Resource inputs for SAM model:
# Must be byte strings
wfd = ssc.data_create()
ssc.data_set_number(wfd, b'lat', user_inputs['project_latitude'])
ssc.data_set_number(wfd, b'lon', user_inputs['project_longitude'])
ssc.data_set_number(wfd, b'tz', timezone)
ssc.data_set_number(wfd, b'elev', elevation)
ssc.data_set_array(wfd, b'year', site_generation.index.year)
ssc.data_set_array(wfd, b'month', site_generation.index.month)
ssc.data_set_array(wfd, b'day', site_generation.index.day)
ssc.data_set_array(wfd, b'hour', site_generation.index.hour)
ssc.data_set_array(wfd, b'minute', site_generation.index.minute)
ssc.data_set_array(wfd, b'dn', site_generation['DNI'])
ssc.data_set_array(wfd, b'df', site_generation['DHI'])
ssc.data_set_array(wfd, b'wspd', site_generation['Wind Speed'])
ssc.data_set_array(wfd, b'tdry', site_generation['Temperature'])

# Create SAM compliant object  
dat = ssc.data_create()
ssc.data_set_table(dat, b'solar_resource_data', wfd)
ssc.data_free(wfd)

# Specify the system Configuration
# Set system capacity in MW
# system_capacity = 4

ssc.data_set_number(dat, b'system_capacity', user_inputs['installed_capacity'])
# Set DC/AC ratio (or power ratio). See https://sam.nrel.gov/sites/default/files/content/virtual_conf_july_2013/07-sam-virtual-conference-2013-woodcock.psite_generation
ssc.data_set_number(dat, b'dc_ac_ratio', user_inputs['dc_ac_ratio'])
# Set tilt of system in degrees
ssc.data_set_number(dat, b'tilt', user_inputs['tilt'])
# Set azimuth angle (in degrees) from north (0 degrees)
ssc.data_set_number(dat, b'azimuth', user_inputs['azimuth'])
# Set the inverter efficency
ssc.data_set_number(dat, b'inv_eff', user_inputs['inv_eff'])
# Set the system losses, in percent
ssc.data_set_number(dat, b'losses', user_inputs['losses'])
# Specify fixed tilt system (0=Fixed, 1=Fixed Roof, 2=1 Axis Tracker, 3=Backtracted, 4=2 Axis Tracker)
ssc.data_set_number(dat, b'array_type', user_inputs['array_type'])
# Set ground coverage ratio
ssc.data_set_number(dat, b'gcr', user_inputs['gcr'])
# Set constant loss adjustment
ssc.data_set_number(dat, b'adjust:constant', user_inputs['adjust:constant'])

# execute and put generation results back into dataframe
mod = ssc.module_create(b'pvwattsv5')
ssc.module_exec(mod, dat)
site_generation['Generation (MW)'] = np.array(ssc.data_get_array(dat, b'gen'))

# free the memory
ssc.data_free(dat)
ssc.module_free(mod)

dc_capacity_factor = site_generation['Generation (MW)'].sum() / (525600/int(interval) * user_inputs['installed_capacity'])

print(f"Total electricity generation at plant location for 2020 is: {site_generation['Generation (MW)'].sum()} MWh")
print(f"Capacity factor is round({dc_capacity_factor}, 3)")
site_generation.head(50)

 0.00 %  @ 0
 0.67 %  @ 175
 1.34 %  @ 350
 2.00 %  @ 525
 2.67 %  @ 700
 3.33 %  @ 875
 4.00 %  @ 1050
 4.67 %  @ 1225
 5.33 %  @ 1400
 6.00 %  @ 1575
 6.66 %  @ 1750
 7.33 %  @ 1925
 7.99 %  @ 2100
 8.66 %  @ 2275
 9.33 %  @ 2450
 9.99 %  @ 2625
10.66 %  @ 2800
11.32 %  @ 2975
11.99 %  @ 3150
12.66 %  @ 3325
13.32 %  @ 3500
13.99 %  @ 3675
14.65 %  @ 3850
15.32 %  @ 4025
15.99 %  @ 4200
16.65 %  @ 4375
17.32 %  @ 4550
17.98 %  @ 4725
18.65 %  @ 4900
19.32 %  @ 5075
19.98 %  @ 5250
20.65 %  @ 5425
21.31 %  @ 5600
21.98 %  @ 5775
22.64 %  @ 5950
23.31 %  @ 6125
23.98 %  @ 6300
24.64 %  @ 6475
25.31 %  @ 6650
25.97 %  @ 6825
26.64 %  @ 7000
27.31 %  @ 7175
27.97 %  @ 7350
28.64 %  @ 7525
29.30 %  @ 7700
29.97 %  @ 7875
30.64 %  @ 8050
31.30 %  @ 8225
31.97 %  @ 8400
32.63 %  @ 8575
33.30 %  @ 8750
Total electricity generation at plant location for 2020 is: 187467.0637221406 MWh
Capacity factor is round(0.21400349739970387, 3)


,Year,Month,Day,Hour,Minute,GHI,DHI,DNI,Wind Speed,Temperature,Solar Zenith Angle,Generation (MW)
2020-01-01 00:00:00,2020,1,1,0,30,0,0,0,1.1,3.7,165.15,0.000000
2020-01-01 01:00:00,2020,1,1,1,30,0,0,0,1.2,3.1,155.28,0.000000
2020-01-01 02:00:00,2020,1,1,2,30,0,0,0,1.4,2.3,143.48,0.000000
2020-01-01 03:00:00,2020,1,1,3,30,0,0,0,1.4,1.9,131.27,0.000000
2020-01-01 04:00:00,2020,1,1,4,30,0,0,0,1.3,2.1,119.07,0.000000
2020-01-01 05:00:00,2020,1,1,5,30,0,0,0,1.0,2.6,107.13,0.000000
2020-01-01 06:00:00,2020,1,1,6,30,0,0,0,0.8,3.0,95.65,0.000000
2020-01-01 07:00:00,2020,1,1,7,30,67,29,417,0.9,4.1,84.75,7.523847
2020-01-01 08:00:00,2020,1,1,8,30,243,46,770,1.2,6.2,75.16,30.618437
2020-01-01 09:00:00,2020,1,1,9,30,399,57,880,1.6,8.8,67.11,48.993471


### **Section 6. Find name of nearest node to your latitude and longitude**

#### **Purpose**

This section provides a quick way to extract the name of the nearest load node to your project's proposed latitude and longitude.

#### **Directions**

Run code chunk and view name of node and its lat/lon as output. These values are used to show where the nearest node is located on energyexplorer.io's UI.

In [50]:
nodes_locations = pd.read_csv('data/LMPLocations.csv')

# find absolute value of user input lat/lon and node dataset lat/lon
# and subtract data lat/lon from user input lat/lons to get differences from user supplied lat lons
nodes_locations['latitude difference'] = abs(user_inputs['project_latitude']) - nodes_locations['latitude'].abs()
nodes_locations['longitude difference'] = abs(user_inputs['project_longitude']) - nodes_locations['longitude'].abs()

# add up absolute value of differences to find total difference then df by total difference to find location with the smallest total difference (the nearest node)
nodes_locations['total difference'] = nodes_locations['latitude difference'].abs() + nodes_locations['longitude difference'].abs() 
nodes_locations = nodes_locations.sort_values(by=['total difference']).query("type == 'Load Node'")

#extra desired node variables
node_request = nodes_locations['name'].iat[0]
node_request_lat = nodes_locations['latitude'].iat[0]
node_request_lon = nodes_locations['longitude'].iat[0]

#show full dataframe
print(f"The nearest load node is {node_request} with a latitude of {node_request_lat} and a longitude of {node_request_lon}")
nodes_locations.head()

The nearest load node is GALE_1_N001 with a latitude of 34.8595 and a longitude of -116.87


,name,type,latitude,longitude,latitude difference,longitude difference,total difference
1810,GALE_1_N001,Load Node,34.8595,-116.87,0.358537,0.070754,0.429291
1814,COLWATER_1_N001,Load Node,34.8602,-116.85,0.357837,0.090754,0.448591
1815,COLWATER_1_N002,Load Node,34.8602,-116.85,0.357837,0.090754,0.448591
1825,TORTILLA_1_N007,Load Node,34.8745,-117.06,0.343537,-0.119246,0.462783
1826,TORTILLA_1_N001,Load Node,34.8745,-117.06,0.343537,-0.119246,0.462783


### **Section 7. Import historic locational marginal price data from HOLLISTR_1_N101 node for 2020**

#### **Purpose**

This section downloads a locally hosted csv with historical locational marginal price data from HOLLISTR_1_N101 node for 2020. In the future this section will dynamically importing historic locational marginal price data from the nearest node location via the CAISO OASIS API.

#### **Directions**

Run code chunk and view name of node and its lat/lon as output.

In [66]:
# Import HOLLISTR_1_N101 node for 2020 data
lmp_only = pd.read_csv('data/LMP_Historic_Price.csv')

# Prepare dataset for use
lmp_only['INTERVALENDTIME_GMT'] = lmp_only['INTERVALENDTIME_GMT'].str.replace('T',' ')
lmp_only['INTERVALENDTIME_GMT'] = lmp_only['INTERVALENDTIME_GMT'].str.replace('-00:00','')
lmp_only['INTERVALENDTIME_GMT'] = pd.to_datetime(lmp_only['INTERVALENDTIME_GMT'], format="%Y-%m-%d %H:%M:%S")
lmp_only = lmp_only[['INTERVALENDTIME_GMT','OPR_HR', 'NODE', 'LMP_TYPE', 'MW']].sort_values(by='INTERVALENDTIME_GMT')

# Preview of dataset
lmp_only.head(12)

,INTERVALENDTIME_GMT,OPR_HR,NODE,LMP_TYPE,MW
1,2020-01-01 09:00:00,1,HOLLISTR_1_N101,LMP,33.17687
2,2020-01-01 10:00:00,2,HOLLISTR_1_N101,LMP,30.89133
0,2020-01-01 11:00:00,3,HOLLISTR_1_N101,LMP,32.20839
3,2020-01-01 12:00:00,4,HOLLISTR_1_N101,LMP,31.34873
7,2020-01-01 13:00:00,5,HOLLISTR_1_N101,LMP,30.93713
16,2020-01-01 14:00:00,6,HOLLISTR_1_N101,LMP,33.39587
8,2020-01-01 15:00:00,7,HOLLISTR_1_N101,LMP,34.19298
18,2020-01-01 16:00:00,8,HOLLISTR_1_N101,LMP,32.92771
19,2020-01-01 17:00:00,9,HOLLISTR_1_N101,LMP,29.57575
20,2020-01-01 18:00:00,10,HOLLISTR_1_N101,LMP,23.55888


### **Section 8. Join site_generation (energy generation output at site) and lmp_only (historical marginal price at nearest wholesale node) on datetime**

#### **Purpose**

This section joins site_generation and lmp_only into one dataframe on their datetimes.

#### **Directions**

Run code chunk and view combined dataframe as well as how many rows were dropped due to some missing datetime values between the two dataframes.

In [67]:
# Join generation and lmp dataframes on datetime

# drop datetime index and make into column named index
site_generation.reset_index(inplace=True)

# rename time column to be be index so join can be performed using index column
lmp_only = lmp_only.rename(columns={'INTERVALENDTIME_GMT': 'index'})

# merge lmp_only and site_generation on index
# note: rows where date and time do not match will be dropped using inner join leading to a dataframe that is less than 8760 lines
lmp_generation_combined = pd.merge(site_generation, lmp_only, how='inner', on='index')

# print out the number of rows dropped to see how much mismatch there was between dfs
number_of_dropped_rows = 8760 - lmp_generation_combined.shape[0]

#reduce size of dataframe to key components
lmp_generation_combined = lmp_generation_combined[['index', 'Generation (MW)', 'NODE', 'MW']]

print(f'{number_of_dropped_rows} rows were droped')
lmp_generation_combined.head(10)

9 rows were droped


,index,Generation (MW),NODE,MW
0,2020-01-01 09:00:00,48.993471,HOLLISTR_1_N101,33.17687
1,2020-01-01 10:00:00,61.175904,HOLLISTR_1_N101,30.89133
2,2020-01-01 11:00:00,63.866039,HOLLISTR_1_N101,32.20839
3,2020-01-01 12:00:00,54.833894,HOLLISTR_1_N101,31.34873
4,2020-01-01 13:00:00,51.654257,HOLLISTR_1_N101,30.93713
5,2020-01-01 14:00:00,47.043224,HOLLISTR_1_N101,33.39587
6,2020-01-01 15:00:00,23.651575,HOLLISTR_1_N101,34.19298
7,2020-01-01 16:00:00,5.346025,HOLLISTR_1_N101,32.92771
8,2020-01-01 17:00:00,0.000000,HOLLISTR_1_N101,29.57575
9,2020-01-01 18:00:00,0.000000,HOLLISTR_1_N101,23.55888


### **Section 9. Run script to simulate additional earnings from storing generation and selling to grid during higher LMP hours**

#### **Directions**

Run code chunk and view additional earning contributions per year from the battery.

In [73]:
##########################################################
#### Create lists to store and test dataframe outputs ####
##########################################################

all_dfs = []

##########################################################
###### Add date and hour values to columns in df #########
##########################################################

# create a new column using index column which has only the day, month, and year on each row
lmp_generation_combined['index_day_only'] = pd.to_datetime(lmp_generation_combined['index']).dt.date

# create a new column using index column which has only the time
lmp_generation_combined['index_hour_only'] = pd.to_datetime(lmp_generation_combined['index']).dt.hour

# split main dataframe into 365 subdataframes each with the same index_day_only value
daily_energy_dfs = {}
for idx, v in enumerate(lmp_generation_combined['index_day_only'].unique()):
    daily_energy_dfs[f'df{idx}'] = lmp_generation_combined.loc[lmp_generation_combined['index_day_only'] == v]

# loop through the list of 365 dataframes, 1 dataframe at a time
for day in np.arange(0, len(daily_energy_dfs), 1):
    
    # loop through the list of 365 dataframes, 1 dataframe at a time. assign current dataframe to current_df
    current_df = daily_energy_dfs[f'df{day}'].loc[:]

    # isolate df with only hours in the day where (1) there is no solar generation (2) the hour of the day is past 12 | and sort df with highest priced hours on top
    top_values_current_day = current_df[current_df['Generation (MW)'] == 0].sort_values('MW', ascending = False).query("index_hour_only > 12")

    # Sort by lowest day ahead price on top to highest price on bottom. We'll assume historical data is exactly representative of day ahead hourly prices.
    # Filter out non-producing hours as we assume battery system can only be filled by the solar system. 
    # These are the hours we'll want to be filling the battery.
    bot_values_current_day = current_df[current_df['Generation (MW)'] != 0].sort_values('MW')

    # Assign current battery charge to 0 at beginning of the day
    # Assume battery is discharged down to 1 at night (cannot be 0 or battery_discharge_hour_counter will not function properly)
    current_battery_charge = .00001

    # Calculate earnings per hour if plant had no battery
    current_df['earnings_per_hour'] = current_df['Generation (MW)'] * current_df['MW']

    # Iterate through each row in bot_values_current_day dataframe
    for hour in reversed(np.arange(0, len(bot_values_current_day), 1)):

        # Check if battery is below capacity
        # If battery is fully charged for the day, this loop will end
        if current_battery_charge < user_inputs['battery_capacity']:

            # calculate what hour in the evening the battery will discharge
            # for example: if the battery has been charged up to 70 MWh out of 400 MWh of capacity and the max discharge rate is 100 MW per hour then the current 70 MWh would be discharged in the first hour
            # for example: if the battery has been charged up to 130 MWh out of 400 MWh of capacity and the max discharge rate is 100 MW per hour then the first 100 MWs would be discharged in the first hour and the remaining 30 MWs in the second
            battery_discharge_hour_counter = int(np.ceil(current_battery_charge / user_inputs['max_discharge_rate'])) - 1

            # Conditionally check if the price per MW during the hour that this generation would be discharged to the grid is higher than the price that is currently being asking for
            if top_values_current_day['MW'].iat[battery_discharge_hour_counter] > bot_values_current_day['MW'].iat[hour]:

                # If battery is below capacity, conditionally decide how much to fill the battery

                # If total generation in that hour is more than max charging rate, then charge battery to max charge rate
                if bot_values_current_day['Generation (MW)'].iat[hour] > user_inputs['max_charge_rate']:

                    add_charge = user_inputs['max_charge_rate']

                # Else charge the battery with that hour's solar system generation (which should be less than or equal to user_inputs['max_charge_rate'])
                else:

                    add_charge = bot_values_current_day['Generation (MW)'].iat[hour]

                # Subtract the amount of charge added from the Generation for that hour
                bot_values_current_day['Generation (MW)'].iat[hour] = bot_values_current_day['Generation (MW)'].iat[hour] - add_charge

                # Update current battery charge with charge added
                current_battery_charge = current_battery_charge + add_charge

                # Check to see if battery capacity was exceeded (overflow charge) by adding new charge to current battery charge
                if current_battery_charge > user_inputs['battery_capacity']:

                    # Calculate overflow charge (the amount of excess charge 'sent' to the battery)
                    extra_charge = current_battery_charge - user_inputs['battery_capacity']

                    # Set current battery charge to battery_capacity by subtracting the extra charge from the total current_battery_charge
                    current_battery_charge = current_battery_charge - extra_charge

                    # Assign the overflow charge to be the generation sent to the grid for that hour. This will be the first hour on the bot_values_current_day dataframe that energy will be immediately sold to the grid
                    bot_values_current_day['Generation (MW)'].iat[hour] = extra_charge

        else:

            break
    

    #Overwrite current_df generation values with bot_values_current_day to reflect energy sent to battery instead of grid
    current_df = current_df.set_index('index')
    current_df.update(bot_values_current_day.set_index('index'))
    current_df = current_df.reset_index()

    ###############################################
    ### Calculate solar revenue for current day ###
    ###############################################

    # Calculate the earning per hour for solar only
    current_df['earnings_per_hour_solar'] = current_df['Generation (MW)'] * current_df['MW']

    # Assign total revenue for that day from solar as an object
    #solar_revenue_sum = bot_values_current_day['earnings_per_hour_solar'].sum()

    #################################################
    ### Calculate battery revenue for current day ###
    #################################################

    #update battery discharge counter again to account for last charge added to battery
    battery_discharge_hour_counter = int(np.ceil(current_battery_charge / user_inputs['max_discharge_rate']))

    add_charge_current_day = current_df[current_df['Generation (MW)'] == 0].sort_values('MW', ascending=False).query("index_hour_only > 12").nlargest(n= battery_discharge_hour_counter, columns = 'MW')
    
    if current_battery_charge > user_inputs['max_discharge_rate'] and current_battery_charge < user_inputs['battery_capacity']:

        add_charge_current_day['battery_discharge'] = user_inputs['max_discharge_rate']

        add_charge_current_day['battery_discharge'].iat[-1] = current_battery_charge % user_inputs['max_discharge_rate']

    elif current_battery_charge == user_inputs['battery_capacity']:

        add_charge_current_day['battery_discharge'] = user_inputs['max_discharge_rate']

    else:
        
        add_charge_current_day['battery_discharge'] = current_battery_charge
    
    add_charge_current_day['earnings_from_battery'] = add_charge_current_day['battery_discharge'] * add_charge_current_day['MW']

    # create final df with solar earnings and battery earnings by hour
    current_df = current_df.merge(add_charge_current_day[['index','battery_discharge','earnings_from_battery']], on='index', how='left')
    
    all_dfs.append(current_df)

generation_with_battery = pd.concat(all_dfs).fillna(0)

generation_with_battery['earnings_from_battery_and_solar'] = generation_with_battery['earnings_per_hour_solar'] + generation_with_battery['earnings_from_battery']

additional_earnings_from_battery = round(generation_with_battery['earnings_from_battery_and_solar'].sum() - generation_with_battery['earnings_per_hour'].sum(), 2)

print(f"By adding a battery the plant earns an additional ${additional_earnings_from_battery} per year")

By adding a battery the plant earns an additional $445578.7 per year


### **Section 10. Calculate remaining parameters for LCOE and calculate LCOE**

#### **Purpose**

In this section we calculate the following parameters used in the LCOE formula:

Calculated paramaters are:
- Annual operating hours (calculated by shaped of post-merge generation and price dataframes) = annual_operating_hours
- CRF = crf 

The calculated parameters will be needed in addtion to the following previously user supplied parameters in Section 3:
- Plant capacity
- Plant lifetime
- Real discount rate
- Tax credit

And the following previously calculated parameters:
- DC capacity factor (year 1) = dc_capacity_factor

#### **Directions**

Run code chunk to calculate parameters and see LCOE output

In [89]:
# calculate the number of annual operating hours
annual_operating_hours = 8760 - number_of_dropped_rows

# crf factor used to discount project cashflows over lifetime
crf = (user_inputs['real_discount_rate']/100 * (1 + user_inputs['real_discount_rate']/100) ** user_inputs['project_lifetime']) / ((1 + user_inputs['real_discount_rate']/100) ** user_inputs['project_lifetime'] - 1)

# calculate lcoe
lcoe = round((((user_inputs['capital_cost_per_mwh']*(1-user_inputs['tax_credit']/100)) * user_inputs['installed_capacity'] * crf) + (user_inputs['fixed_per_mwh'] * user_inputs['installed_capacity'])) / (user_inputs['installed_capacity'] * dc_capacity_factor * annual_operating_hours),2)

print(f"Levelized cost of energy is ${lcoe}")

Levelized cost of energy is $32.18


### **Section 11. Calculate average earnings in dollars per MWh and net average earnings in dollars per MWh after subtracting LCOE**

#### **Purpose**

In this section we calculate total annual generation (which accounts for the reduction in the size of the dataframe after the merge) and total annual earnings a parameters to calculate average earning per MWh

#### **Directions**

Run code chunk

In [97]:
# multiply generation per hour ['Generation (MW)'] by price per hour [MW] and put result in new column ['earnings_per_hour']
lmp_generation_combined['earnings_per_hour'] = lmp_generation_combined['Generation (MW)'] * lmp_generation_combined['MW']

# find sum of hourly generation and hourly earnings to find average earning per hour
total_annual_generation = lmp_generation_combined['Generation (MW)'].sum()
total_annual_earnings = lmp_generation_combined['earnings_per_hour'].sum()

# divide total annual earnings by total annual generation to find average earning per hour
average_earnings_per_mwh = total_annual_earnings/total_annual_generation

net_average_earnings_per_mwh = round(average_earnings_per_mwh - lcoe, 3)

print(f"Average net earnings is ${net_average_earnings_per_mwh} per MWh")

Average net earnings is $-4.028 per MWh


### **Section 12. Calculate lifetime value of solar + battery plant**

#### **Directions**

Run code chunk


In [102]:
lifetime_value_df = pd.DataFrame()

selected_node = lmp_only['NODE'].iat[0]

lifetime_value_df['Year'] = np.arange(start= int(year), stop = int(year) + user_inputs['project_lifetime'], step= 1)
lifetime_value_df['Years from Start'] = np.arange(start= 0, stop = user_inputs['project_lifetime'], step= 1)
lifetime_value_df['LCOE ($/MWh)'] = round(lcoe, 2)
lifetime_value_df[f'Income from nearest node {selected_node} ($/MWh)'] = average_earnings_per_mwh
lifetime_value_df['Inflation Adjusted Subsidy ($/MWh)'] = (user_inputs['subsidy']/((1+(user_inputs['real_discount_rate']/100))**lifetime_value_df['Years from Start']))
lifetime_value_df['Subsidized Income ($/MWh)'] = round(lifetime_value_df[f'Income from nearest node {selected_node} ($/MWh)'] + lifetime_value_df['Inflation Adjusted Subsidy ($/MWh)'], 2)
lifetime_value_df['NPV Annual Income ($/Year)'] = round(((lifetime_value_df['Subsidized Income ($/MWh)'] - lifetime_value_df['LCOE ($/MWh)'])* total_annual_generation)/((1+(user_inputs['real_discount_rate']/100))**lifetime_value_df['Years from Start']), 2)

project_lifetime_value = round(lifetime_value_df['NPV Annual Income ($/Year)'].sum(), 3)

print(f"Lifetime value of project is ${project_lifetime_value}")
lifetime_value_df

Lifetime value of project is $-11177984.69


,Year,Years from Start,LCOE ($/MWh),Income from nearest node HOLLISTR_1_N101 ($/MWh),Inflation Adjusted Subsidy ($/MWh),Subsidized Income ($/MWh),NPV Annual Income ($/Year)
0,2020,0,32.18,28.152469,0.0,28.15,-755338.55
1,2021,1,32.18,28.152469,0.0,28.15,-719370.05
2,2022,2,32.18,28.152469,0.0,28.15,-685114.33
3,2023,3,32.18,28.152469,0.0,28.15,-652489.84
4,2024,4,32.18,28.152469,0.0,28.15,-621418.90
5,2025,5,32.18,28.152469,0.0,28.15,-591827.52
6,2026,6,32.18,28.152469,0.0,28.15,-563645.26
7,2027,7,32.18,28.152469,0.0,28.15,-536805.01
8,2028,8,32.18,28.152469,0.0,28.15,-511242.86
9,2029,9,32.18,28.152469,0.0,28.15,-486897.97


### 10. Build plot of NPV of cashflow streams from plant through time


In [103]:
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.graph_objs as go


#Create graph object Figure object with data
fig = go.Figure(data = go.Bar(x = lifetime_value_df.index, y = lifetime_value_df['NPV Annual Income ($/Year)'], marker_color='#0d6efd'))

#Update layout for graph object Figure
fig.update_layout(barmode='stack', 
                  title_text = f"NPV Annual Income ($/Year) At Lat: {user_inputs['project_latitude']}, Lon: {user_inputs['project_longitude']}",
                  xaxis_title = '($/Year)',
                  yaxis_title = 'Year',
                  yaxis_tickprefix = '$', 
                  yaxis_tickformat = ',')

fig

#plotly_plot_obj = plot({'data': fig}, output_type='div')
